In [ ]:
# Código para abrir los diccionarios .pkl

import pickle as pkl
archivo_npz = ("archivo.pkl")
with open(archivo_npz, 'rb') as f:
	archivo = pkl.load(f, encoding='latin1')

# Código para abrir los diccionarios .npz

import numpy as np
archivo_npz = np.load("archivo.npz", allow_pickle=True)

In [ ]:
# Representar el modelo SMPL utilizando Aitviewer

from aitviewer.models.smpl import SMPLLayer
from aitviewer.renderables.smpl import SMPLSequence
from aitviewer.viewer import Viewer
from aitviewer.renderables.spheres import Spheres
import numpy as np

smpl_layer = SMPLLayer(model_type="smpl", gender="male")

# Crear una SMPLSequence estática
poses = np.zeros([1, smpl_layer.bm.NUM_BODY_JOINTS * 3]) # 1 frame (ya que es estatico), 72 joints (24 joint x 3 grados de libertad)
smpl_seq = SMPLSequence(poses, smpl_layer)

viewer = Viewer()
viewer.scene.add(smpl_seq)
viewer.run()

In [ ]:
# Representar una secuencia proveniente de AMASS

from aitviewer.models.smpl import SMPLLayer
from aitviewer.renderables.smpl import SMPLSequence
from aitviewer.viewer import Viewer
from aitviewer.configuration import CONFIG as C
import os
import numpy as np

c = (155 / 255, 87 / 255, 155 / 255, 1)
smpl_seq = SMPLSequence.from_amass(
        npz_data_path=os.path.join(C.datasets.amass, r"EyesJapanDataset\EyesJapanDataset\shiono\greeting-04-salute (head)-shiono_poses.npz"),
        fps_out=60.0,
        name="AMASS_1", 
        show_joint_angles=True,
        color = c
    )

viewer = Viewer()
viewer.scene.add(smpl_seq)
viewer.run()

In [ ]:
# Código para convertir archivos .npz a .pkl

import os
import numpy as np
import pickle

def process_npz_file(npz_file_path, input_folder, output_folder):
    data = np.load(npz_file_path)
    extracted_data = {
        'gender': data['gender'],
        'betas': data['betas'],
        'poses': data['poses'],
        'frame_rate': data['mocap_framerate'], # Si no funciona prueba con 'mocap_frame_rate'
        'trans': data['trans'],
    }

    relative_path = os.path.relpath(npz_file_path, start=input_folder)

    output_folder_path = os.path.join(output_folder, os.path.dirname(relative_path))
    os.makedirs(output_folder_path, exist_ok=True)

    pkl_file_path = os.path.join(output_folder_path, os.path.basename(npz_file_path).replace('.npz', '_pkl_version.pkl'))

    with open(pkl_file_path, 'wb') as pkl_file:
        pickle.dump(extracted_data, pkl_file)

def process_folder(input_folder, output_folder):
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.npz'):
                npz_file_path = os.path.join(root, file)
                process_npz_file(npz_file_path, input_folder, output_folder)

# Ruta de la carpeta principal que contiene las subcarpetas
main_folder_path = r'C:\Users\danie\OneDrive\Escritorio\TFG\TFG_entorno\AMASS_Clasificado\HDM05\HDM05'

# Carpeta de salida para los archivos pkl
output_folder_path = r'c:\Users\danie\OneDrive\Escritorio\TFG\TFG_entorno\archivospkl\HDM05\HDM05'

process_folder(main_folder_path, output_folder_path)

In [ ]:
# Cópdigo de síntesis

import glob
import os
import cv2
import sys
import numpy as np
import chumpy as ch
import pickle as pkl

# TODO
# Añadir las rutas de los archivos SMPL
sys.path.append("C:\\Users\\danie\\OneDrive\\Escritorio\\SMPL_python_v.1.1.0\\smpl")
sys.path.append("C:\\Users\\danie\\OneDrive\\Escritorio\\SMPL_python_v.1.1.0\\smpl\\smpl_webuser")

from smpl_webuser.serialization import load_model
from smpl_webuser.lbs import global_rigid_transformation

#model_path = "C:\\Users\\danie\\OneDrive\\Escritorio\\SMPL_python_v.1.1.0\\smpl\\models\\smplh\\%s_smplh.pkl" # Para smplh
model_path = "C:\\Users\\danie\\OneDrive\\Escritorio\\SMPL_python_v.1.1.0\\smpl\\models\\smpl\\%s.pkl" # Para smpl
model_male = load_model(model_path % "male")
model_female = load_model(model_path % "female")

Jdirs_male = np.dstack([model_male.J_regressor.dot(model_male.shapedirs[:,:,i]) for i in range(10)])
Jdirs_female = np.dstack([model_female.J_regressor.dot(model_male.shapedirs[:,:,i]) for i in range(10)])

# TODO
# Please modify here to specify which vertices to use
VERTEX_IDS = [6740,5763,4560,5213,4361,6060,3021,3498]
# rfoot, lfoot, rlowleg, llowleg, rhighleg, lhighleg, back, chest

# TODO
TARGET_FPS = 60

# TODO
# Please modify here to specify which SMPL joints to use
SMPL_IDS = [1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

# Extrae la categoría de movimiento del nombre del archivo
def extract_movement_category(file_name):
    keywords_stand = ['stand', 'neutral', 'rest', 'static', 'wave', 'shake_arm', 'shake_shoulder', 'gesture_etc-05-touch nose']
    keywords_walk = ['walk', 'walking','treadmill']
    keywords_run = ['run', 'jog', 'jogging', 'running']
    keywords_sit = [ 'sit', 'sitting', 'sitdown', 'situp', 'chair']
    keywords = [keywords_stand, keywords_walk, keywords_run, keywords_sit]

    for keyword_class in keywords:
        for keyword in keyword_class:
            if keyword in file_name.lower():
                return keyword_class[0]
            
    return 'unknown'

# Get orientation and acceleration from list of 4x4 matrices, and vertices
def get_ori_accel(A_global_list, vertex, frame_rate):
    orientation = []
    acceleration = []

    for a_global in A_global_list:
        ori_left_foot = a_global[10][:3, :3].r
        ori_right_foot = a_global[11][:3, :3].r
        ori_left_lowleg = a_global[4][:3, :3].r
        ori_right_lowleg = a_global[5][:3, :3].r
        ori_left_highleg = a_global[1][:3, :3].r
        ori_right_highleg = a_global[2][:3, :3].r
        ori_chest = a_global[9][:3, :3].r
        ori_root = a_global[3][:3, :3].r

        ori_tmp = []
        ori_tmp.append(ori_left_foot)
        ori_tmp.append(ori_right_foot)
        ori_tmp.append(ori_left_lowleg)
        ori_tmp.append(ori_right_lowleg)
        ori_tmp.append(ori_left_highleg)
        ori_tmp.append(ori_right_highleg)
        ori_tmp.append(ori_chest)
        ori_tmp.append(ori_root)
        
        orientation.append(np.array(ori_tmp))

    time_interval = 1.0 / frame_rate
    total_number = len(A_global_list)
    for idx in range(1, total_number-1):
        vertex_0 = vertex[idx-1].astype(float) # 6*3
        vertex_1 = vertex[idx].astype(float)
        vertex_2 = vertex[idx+1].astype(float)
        accel_tmp = (vertex_2 + vertex_0 - 2*vertex_1) / (time_interval*time_interval)

        acceleration.append(accel_tmp)

    return orientation[1:-1], acceleration


def compute_imu_data(gender, betas, poses, frame_rate):
    if gender == 'male':
        Jdirs = Jdirs_male
        model = model_male
    else:
        Jdirs = Jdirs_female
        model = model_female

    betas[:] = 0
    J_onbetas = ch.array(Jdirs).dot(betas) + model.J_regressor.dot(model.v_template.r)

    A_global_list = []
    print( 'Longitud del archivo: %d' % (len(poses)/1) )
    for idx, p in enumerate(poses):
        if len(p) != 72:
            raise ValueError("La longitud de la pose debe ser 72.")
        (_, A_global) = global_rigid_transformation(p, J_onbetas, model.kintree_table, xp=ch)
        A_global_list.append(A_global)
    
    vertex = []
    for idx, p in enumerate(poses):
        model.pose[:] = p
        model.betas[:] = 0
        model.betas[:10] = betas
        tmp =  model.r[VERTEX_IDS]
        vertex.append(tmp) # 6*3
            

    orientation, acceleration = get_ori_accel(A_global_list, vertex, frame_rate)

    return orientation, acceleration


def findNearest(t, t_list):
    list_tmp = np.array(t_list) - t
    list_tmp = np.abs(list_tmp)
    index = np.argsort(list_tmp)[:2]
    return index
	

# Turn MoCap data into 60FPS
def interpolation_integer(poses_ori, fps):
    poses = []
    n_tmp = int(fps / TARGET_FPS)
    poses_ori = poses_ori[::n_tmp]
    
    for t in poses_ori:
        poses.append(t)

    return poses

def interpolation(poses_ori, fps):
    poses = []
    total_time = len(poses_ori) / fps
    times_ori = np.arange(0, total_time, 1.0 / fps)
    times = np.arange(0, total_time, 1.0 / TARGET_FPS)
    
    for t in times:
        index = findNearest(t, times_ori)
        a = poses_ori[index[0]]
        t_a = times_ori[index[0]]
        b = poses_ori[index[1]]
        t_b = times_ori[index[1]]

        if t_a == t: 
            tmp_pose = a
        elif t_b == t:
            tmp_pose = b
        else:
            tmp_pose = a + (b-a)*((t_b-t)/(t_b-t_a)) 
        poses.append(tmp_pose)

    return poses


# Extract pose parameter from pkl_path, save to res_path
def generate_data(pkl_path, res_path):
    if os.path.exists(res_path):
        return

    with open(pkl_path, 'rb') as fin:
        data_in = pkl.load(fin, encoding='latin1')
    
    data_out = {}
    data_out['gender'] = data_in['gender']
    data_out['betas'] = np.array(data_in['betas'][:10])
    data_in_poses = data_in['poses'][:, :72] # De esta forma nos quedamos solo con los 72 primeros parámetros, corespondientes al modelo SMPL, en lugar de usar el modelo SMPLX (con el cual el codigo no seria compatible)

    # In case the original frame rates (eg 40FPS) are different from target rates (60FPS) 
    fps_ori = data_in['frame_rate']
    if (fps_ori % TARGET_FPS) == 0:
        data_out['poses'] = interpolation_integer(data_in_poses, fps_ori)
    else:
        data_out['poses'] = interpolation(data_in_poses, fps_ori)

    data_out['ori'], data_out['acc'] = compute_imu_data(data_out['gender'], data_out['betas'], data_out['poses'], TARGET_FPS)
    
    data_out['poses'] = data_out['poses'][1:-1]

    for fdx in range(0, len(data_out['poses'])):
        pose_tmp = []#np.zeros(0)
        for jdx in SMPL_IDS:
            tmp = data_out['poses'][fdx][jdx*3:(jdx+1)*3]
            tmp = cv2.Rodrigues(tmp)[0].flatten().tolist()
            pose_tmp = pose_tmp + tmp

        data_out['poses'][fdx] = []
        data_out['poses'][fdx] = pose_tmp

    data_out_ = {}
    data_out_['ori'] = data_out['ori']
    data_out_['acc'] = data_out['acc']
    file_name = os.path.basename(pkl_path)
    data_out_['gt'] = extract_movement_category(file_name)

    with open(res_path, 'wb') as fout:
            pkl.dump(data_out_, fout)
    print( pkl_path )
    print( res_path )
    print( len(data_out['acc']) )
    print( '' )


def get_relative_path(pkl_file_path, input_folder):
    return os.path.relpath(pkl_file_path, start=input_folder)

def generate_synthetic_data(pkl_file_path, output_folder):
    print(f"Procesando archivo: {pkl_file_path}")
    
    relative_path = get_relative_path(pkl_file_path, pkl_folder_path)

    output_folder_path = os.path.join(output_folder, os.path.dirname(relative_path))
    os.makedirs(output_folder_path, exist_ok=True)

    res_data_path = os.path.join(output_folder_path, os.path.basename(pkl_file_path).replace('.pkl', '_synthetic.pkl'))

    generate_data(pkl_file_path, res_data_path)

def process_pkl_folder(input_folder, output_folder):
    # Puede que sea necesario que se cambie el patrón de búsqueda de archivos pkl, dependiendo de la estructura de la carpeta. En mi caso, los archivos pkl estaban en subcarpetas de 4 niveles de profundidad.
    print(os.path.join(input_folder, '*.pkl'))
    for pkl_file_path in glob.glob(os.path.join(input_folder, '*/*/*/*.pkl'), recursive=True):
        print(f"Procesando archivo: {pkl_file_path}")
        generate_synthetic_data(pkl_file_path, output_folder)

# Ruta de la carpeta principal que contiene los archivos pkl
pkl_folder_path = 'C:\\Users\\danie\\OneDrive\\Escritorio\\TFG\\TFG_entorno\\archivospkl'

# Carpeta de salida para los archivos sintéticos pkl
output_folder_path = 'C:\\Users\\danie\\OneDrive\\Escritorio\\TFG\\TFG_entorno\\datos_sinteticos_amass_definitivos'

if __name__ == '__main__':
    process_pkl_folder(pkl_folder_path, output_folder_path)
    

"""
@article{DIP:SIGGRAPHAsia:2018,
	title = {Deep Inertial Poser: Learning to Reconstruct Human Pose from Sparse Inertial Measurements in Real Time},
    	author = {Huang, Yinghao and Kaufmann, Manuel and Aksan, Emre and Black, Michael J. and Hilliges, Otmar and Pons-Moll, Gerard},
    	journal = {ACM Transactions on Graphics, (Proc. SIGGRAPH Asia)},
    	volume = {37},
    	pages = {185:1-185:15},
    	publisher = {ACM},
    	month = nov,
    	year = {2018},
    	note = {First two authors contributed equally},
    	month_numeric = {11}
}
"""